In [150]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [151]:
data = pd.read_csv("predict.csv")
# Считываем данные

In [152]:
data = data.fillna(0)
print("Количество уникальных элеменов в authors_mean_pagerank - {}".format(len(set(data["authors_mean_pagerank"].tolist()))))
print("Количество уникальных элеменов в authors_mean_productivity - {}".format(len(set(data["authors_mean_productivity"].tolist()))))
print("Количество уникальных элеменов в journal_pagerank - {}".format(len(set(data["journal_pagerank"].tolist()))))

Количество уникальных элеменов в authors_mean_pagerank - 1
Количество уникальных элеменов в authors_mean_productivity - 1
Количество уникальных элеменов в journal_pagerank - 1


In [153]:
# Получается, что столбцы authors_mean_pagerank, authors_mean_productivity и journal_pagerank содержали нули

In [154]:
data = data.drop(["authors_mean_pagerank", "authors_mean_productivity", "journal_pagerank"], axis=1)

In [155]:
# Для удобства поменяем c5 (и назовём y) - наш таргетный столбец, который мы хотим предсказывать с log_authors_mean_sociality
data["y"] = data["c5"]
data = data.drop("c5", axis = 1)

In [156]:
data

,recency,topic_rank,diversity,authors_mean_rank,authors_mean_hindex,authors_mean_gindex,authors_mean_sociality,journal_rank,title_len,abstract_len,n_authors,log_authors_mean_sociality,y
0,16,1.0,-0.425436,500.0,1.0,1.0,6.0,101.0,66,1653,7,1.945910,1.0
1,8,7.0,-0.571967,134.0,1.0,1.0,6.0,24.0,59,1527,7,1.945910,32.0
2,15,19.0,-0.859644,235.0,1.0,1.0,5.0,49.0,169,1554,6,1.791759,5.0
3,6,4.0,-1.054999,1124.0,1.0,1.0,10.0,185.0,130,2121,11,2.397895,7.0
4,14,2.0,-0.476697,346.0,1.0,1.0,2.0,72.0,100,432,3,1.098612,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,25,6.0,-0.330948,528.0,0.0,0.0,6.0,140.0,62,645,7,1.945910,0.0
475,6,12.0,-1.493480,2145.0,0.0,0.0,1.0,416.0,45,714,2,0.693147,0.0
476,24,17.0,-1.466320,551.0,0.0,0.0,4.0,146.0,84,0,5,1.609438,0.0
477,24,6.0,-1.556737,551.0,0.0,0.0,0.0,146.0,56,0,1,0.000000,0.0


In [157]:
X = data.iloc[:, :-1] # Разбиваем данные на X и y
y = data.iloc[:, -1]

In [158]:
from sklearn.model_selection import train_test_split # Разбиваем выборку на X_train и X_test - для первичного анализа
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [159]:
from sklearn import linear_model # Узнаем, какие признаки можно отобрать
clf = linear_model.Lasso(alpha=0.1) # L1-регуляризация отвечает за отбор признаков, так как наказывает модель за большие веса
clf.fit(X_train, y_train)

Lasso(alpha=0.1)

In [160]:
from sklearn.metrics import r2_score
y_pred = clf.predict(X_test)

In [161]:
r2_score(y_test, y_pred) # r2_score не удолетворительная, поэтому думаем над улучшениями

0.331776624561264

In [162]:
clf.coef_ # Убираем признаки, которые обнулились

array([-1.44371216e+00,  3.79897458e-02, -5.22729146e-01, -1.50830012e-02,
       -3.54597930e+00, -0.00000000e+00,  1.38219015e-01, -1.28515488e-02,
       -1.59125576e-02,  3.15699938e-04,  0.00000000e+00, -0.00000000e+00])

In [163]:
data = data.drop(data.columns[[5, 9, 10, 11]], axis = 1) # Удаляем ненужные столбцы

In [164]:
data

,recency,topic_rank,diversity,authors_mean_rank,authors_mean_hindex,authors_mean_sociality,journal_rank,title_len,y
0,16,1.0,-0.425436,500.0,1.0,6.0,101.0,66,1.0
1,8,7.0,-0.571967,134.0,1.0,6.0,24.0,59,32.0
2,15,19.0,-0.859644,235.0,1.0,5.0,49.0,169,5.0
3,6,4.0,-1.054999,1124.0,1.0,10.0,185.0,130,7.0
4,14,2.0,-0.476697,346.0,1.0,2.0,72.0,100,7.0
...,...,...,...,...,...,...,...,...,...
474,25,6.0,-0.330948,528.0,0.0,6.0,140.0,62,0.0
475,6,12.0,-1.493480,2145.0,0.0,1.0,416.0,45,0.0
476,24,17.0,-1.466320,551.0,0.0,4.0,146.0,84,0.0
477,24,6.0,-1.556737,551.0,0.0,0.0,146.0,56,0.0


In [165]:
from sklearn import preprocessing

scaler = preprocessing.MinMaxScaler()

names = data.iloc[:, :-1].columns
d = scaler.fit_transform(data.iloc[:, :-1])

data = pd.DataFrame(d, columns=names)
data["y"] = y
data

,recency,topic_rank,diversity,authors_mean_rank,authors_mean_hindex,authors_mean_sociality,journal_rank,title_len,y
0,0.55,0.000000,0.874534,0.214041,0.888889,0.400000,0.231651,0.190114,1.0
1,0.15,0.315789,0.824677,0.057363,0.888889,0.400000,0.055046,0.163498,32.0
2,0.50,0.947368,0.726796,0.100599,0.888889,0.333333,0.112385,0.581749,5.0
3,0.05,0.157895,0.660327,0.481164,0.888889,0.666667,0.424312,0.433460,7.0
4,0.45,0.052632,0.857092,0.148116,0.888889,0.133333,0.165138,0.319392,7.0
...,...,...,...,...,...,...,...,...,...
474,1.00,0.263158,0.906683,0.226027,0.000000,0.400000,0.321101,0.174905,0.0
475,0.05,0.578947,0.511136,0.918236,0.000000,0.066667,0.954128,0.110266,0.0
476,0.95,0.842105,0.520377,0.235873,0.000000,0.266667,0.334862,0.258555,0.0
477,0.95,0.263158,0.489613,0.235873,0.000000,0.000000,0.334862,0.152091,0.0


In [166]:
X = data.iloc[:, :-1] # Разбиваем данные на X и y
y = data.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [167]:
clf = linear_model.Lasso(alpha=0.1) # L1-регуляризация отвечает за отбор признаков, так как наказывает модель за большие веса
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
r2_score(y_test, y_pred)

0.33637370050731275

In [168]:
# Как мы видим, отбросив некоторые ненужные признаки и нормализовав данные мы увеличили r2_score

In [169]:
# После отбора признаков, фиче-инжиниринга, и теста линейных моделей (Linear Regression с L1 и L2 регуляризацией), продолжаем
# работать с более продвинутыми моделями - с ансаблями (дерево решений показало не высокий результат, а градиентный бустинг
# более плохой, чем случайный лес)

In [173]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor

clf = RandomForestRegressor()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
r2_score(y_test, y_pred)

0.4615521144885272

In [174]:
# Как мы видим - значение r2 0.46 нам не подходит, стоит перебрать правильные гипер-параметры для задачи

In [175]:
from sklearn.model_selection import cross_val_score
print(cross_val_score(clf, X, y, cv=5).mean())

0.47545669339103913


In [176]:
# Импортируем из sklearn GridSearchCV, который поможет перебрать нужные для нас параметры

In [177]:
from sklearn.model_selection import GridSearchCV

param = {
    "n_estimators" : list(range(2, 50)),
    "max_depth" : list(range(3, 8)),
    "criterion" : ['mae', 'mse']
}

grid = GridSearchCV(RandomForestRegressor(), param_grid=param, cv=5)
grid.fit(X_train, y_train)
grid.best_params_

{'criterion': 'mae', 'max_depth': 5, 'n_estimators': 20}

In [230]:
clf = RandomForestRegressor(max_depth = 7, n_estimators = 38, criterion = 'mae')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
r2_score(y_test, y_pred)

0.48882448033966575

In [231]:
print(cross_val_score(clf, X, y, cv=5).mean())

0.5905007295964804


In [240]:
from sklearn.model_selection import KFold
from tqdm import tqdm
cv = KFold(n_splits=6, random_state=42, shuffle=True)

# Сделаем кросс-валидацию через KFold для r2_score
results = []
for train_index, test_index in tqdm(cv.split(y)):
    X_train, X_test = X.iloc[train_index.tolist(), :], X.iloc[test_index.tolist(), :]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    results.append(r2_score(y_test, y_pred))

6it [00:01,  5.15it/s]


In [241]:
np.array(results).mean()

0.7292392581900023

In [ ]:
# Как мы можем понять, на кросс-валидации получаем средний результат r2 = +- 0.7, что является довольно хорошим показателем